In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from pprint import pprint
from dotenv import load_dotenv
from time import sleep
from tqdm import tqdm
import pandas as pd
from dotenv import dotenv_values, set_key
import json
import csv
import os
import requests
import base64


#### You'll need to add client id and secret id stuffs somewhere

In [3]:
def get_refresh_token(client_id, client_secret, redirect_uri, scope):
    # Set up SpotifyOAuth with the required parameters
    sp_oauth = SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        scope=scope,
    )

    # Get the authorization URL
    auth_url = sp_oauth.get_authorize_url()

    print(f"Please go to the following URL to authorize your account: {auth_url}")

    sleep(3)
    # Get the authorization code from the user
    auth_code = input("Enter the authorization code from the URL: ")

    # Exchange the authorization code for access and refresh tokens
    token_info = sp_oauth.get_cached_token(str(auth_code))

    # Extract and print the refresh token
    refresh_token = token_info.get('refresh_token')
    if refresh_token:
        print(f"Your refresh token is: {refresh_token}")
    else:
        print("Failed to retrieve refresh token. Please check your authorization code.")

def refresh_access_token(client_id, client_secret, refresh_token, redirect_uri, scope):
    # Set up SpotifyOAuth with the required parameters
    sp_oauth = SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        scope=scope,
    )

    # Refresh the access token using the refresh token
    token_info = sp_oauth.refresh_access_token(refresh_token)

    # Extract and print the new access token
    access_token = token_info.get('access_token')
    if access_token:
        print(f"Your new access token is: {access_token}")
    else:
        print("Failed to refresh access token. Please check your refresh token.")



# Example usage:

# get_refresh_token(client_id, client_secret, redirect_uri, scope)
# refresh_access_token(client_id, client_secret, refresh_token, redirect_uri, scope)



In [4]:
import pandas as pd

df = pd.read_csv('../data/artists_ids.csv')
df


,id,name
0,5szilpXHcwOqnyKLqGco5j,Faye Webster
1,07EcmJpfAday8xGkslfanE,Kevin Abstract
2,2lciIw4vq0jQqevXA2TEUl,Kara Jackson
3,4tK6Z8fK7Sc9133byjPGIT,MJ Lenderman
4,59RNNqeEfkq3X5pfOQxZ3C,Hovvdy
...,...,...
62923,46fSsw6Lx6s1kvYGnb4pnd,Meemo Comma
62924,3L3ejltt5dmjXkES4YSGKX,Oli XL
62925,7CXSuH40c4n4hVbP1JDBFP,Surgeons Girl
62926,6WPeOlm3f3UGoO1l6OPOUO,Krust


In [5]:
ids = df['id'].tolist()
pprint(ids)


['5szilpXHcwOqnyKLqGco5j',
 '07EcmJpfAday8xGkslfanE',
 '2lciIw4vq0jQqevXA2TEUl',
 '4tK6Z8fK7Sc9133byjPGIT',
 '59RNNqeEfkq3X5pfOQxZ3C',
 '5QeOEXfGjdzdEsUOHODPCw',
 '6VgPyGeGO86DztjK7GCYT3',
 '2uYWxilOVlUdk4oV9DvwqK',
 '4j7DrazfBZLLD0OrVoAtEe',
 '2Ru6erK9heeCeWQDx0sDmR',
 '0pg0Zm8FsGAYy5kdHuBnSo',
 '6lcwlkAjBPSKnFBZjjZFJs',
 '38u18VoGaIwVeSyVoA0eU5',
 '5gspAQIAH8nJUrMYgXjCJ2',
 '2JFTRDi5v7JtqoouVe1z5D',
 '4iuECUfKK1Iulmgxt5MObO',
 '7o1TBmx7Ube5h2Czlam84O',
 '1thbOfXak53dM1Xabq3pmb',
 '46VzbEZDkCQKJX3o4g8gMD',
 '392WuM1Yb4QRI0GG4epyn5',
 '0avMDS4HyoCEP6RqZJWpY2',
 '5QdyldG4Fl4TPiOIeMNpBZ',
 '2y1A93fUkRi2xSbKMwhyzs',
 '48mkLSnW17hfCg0rDjMzs7',
 '40ZNYROS4zLfyyBSs2PGe2',
 '4LEiUm1SRbFMgfqnQTwUbQ',
 '3lWVgSwutPsiJ8Awm7OTKU',
 '5b1DzFl2lTpE9v5fjAZ9kn',
 '73mSg0dykFyhvU96tb5xQV',
 '6YWdHD3R863Apw1hkx3BwC',
 '251UrhgNbMr15NLzQ2KyKq',
 '00FQb4jTyendYWaN8pK0wa',
 '2kGBy2WHvF0VdZyqiVCkDT',
 '0xPoVNPnxIIUS1vrxAYV00',
 '0WCo84qtCKfbyIf1lqQWB4',
 '3u39r3hiQU2sq7VrV38L6x',
 '7vtSUU3zpHeYJfX6BPNrJd',
 

In [6]:
print(client_id)
print(client_secret)

auth_manager = SpotifyClientCredentials(client_id=client_id,
                                        client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)
# print(help(sp))


244c139a02bc4149bc464b367538ca20
f7f6b4447da047659a4f84cfb00f3733


In [7]:

def append_artist_to_csv(filename, data_dict):

    for track in data_dict['items']:
        # if type(song_data['track']) is type(None):
        #     continue
        song = song_data['track']['artists']
        fieldnames = song[0].keys()
        file_exists = os.path.isfile(filename)

        try:
            with open(filename, mode='a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                if not file_exists:
                    writer.writeheader()
                for artist in song:
                    writer.writerow(artist)
        except Exception as e:
            print(song)
            continue



def append_artist_to_json(filename, data_dict):

    try:
        with open(filename, 'r') as f:
            existing_data = json.load(f)
    except FileNotFoundError:
        existing_data = {}

    existing_data.update(data_dict)

    with open(filename, 'w') as f:
        json.dump(existing_data, f)


In [16]:
artist_top_tracks = {}

for id in tqdm(ids):
    try:
        results = sp.artist_top_tracks(artist_id=id, country='US')
        sleep(.2)
        artist_top_tracks[id] = []
        for track in results['tracks']:
            artist_top_tracks[id].append(track['id'])
        append_artist_to_json('artist_top_songs.json', artist_top_tracks)
    except spotipy.SpotifyException as e:
        print(e)
        if e.http_status == 401 and 'access token expired' in e.msg.lower():
            print('Refreshing token')
            refresh_access_token(client_id,
                                 client_secret,
                                 refresh_token,
                                 redirect_uri,
                                 scope)
        if e.http_status == 429:
            print('Sleeping for 30 seconds')
            sleep(30)
            print(e.headers)
    except Exception as e:
        print(e)
        # failed_artist_ids.append(data)
        continue


  0%|          | 0/62928 [00:00<?, ?it/s]

  8%|▊         | 4945/62928 [35:32<6:56:45,  2.32it/s] 


KeyboardInterrupt: 

In [14]:
storage = pd.read_json('artists_top_songs.json', orient='index')
storage



,0,1,2,3,4,5,6,7,8,9
5szilpXHcwOqnyKLqGco5j,71BWZa1liIRyUiuJ3MB66o,5WbfFTuIldjL9x7W6y5l7R,3cxZT78mZDyLsLPJKcTu3U,2VYnlKxB8jpbCeLUjQsebc,0uphGVj1uIticJK6gb2WnO,5tuBenbZ1Or9xENTyOKoGC,0UaqZbfeKuOdPxkyK7782l,7hNPcyHhNYLCtTFa5bxvX1,5sGGlfbqdGlJr3ZMMjODoy,4E3JYij2hnEDDP0Rgwikq6
07EcmJpfAday8xGkslfanE,5JRMqkR82k2fdDEAim9SCN,08lxQMon7L91ykAMi87rgN,6WlMBy1QoNnCOfBXMyPesO,6gSw5vz7dcafda23VqMrnl,7D0LbItAj4QvrUrhig5SoQ,00PNqWHjDUND2R7B4oJ93J,5K7dEG6yDQ981HO7kFMVqm,71y24qPwDLjqSm0ZJT8iqY,2weMF1nCZANujeWOvy73sa,23pPTbWfZqWq3gn1hRiJ7n
2lciIw4vq0jQqevXA2TEUl,5khPTiW1Ufc5uXMEqOaOXP,1HoSyT8MIKRUZ3q3ohEFiK,5YTBLQ5C8cG38Ch41jiJNl,0MjjrdCKDu19Z1obkdkULO,4sriGuSuKyc8mmcoWaNO2b,58DEyGYEzY63sIWOuPlGDl,1SLaD2WxICo8ma3oAfKnRX,5SlwouQoZO7q3dEdqkMBhi,1UklQvWcGtHSvn5TuYO1P8,1qWsVMgZZQWTg2XkSja0OU
4tK6Z8fK7Sc9133byjPGIT,1P9RJszAMDvXfJ40uuchAt,657PpSizWfiabkjgIzEFWa,0ll3IRoSlJ461DTxW4Z7vc,7kTUoMrKA0duKYHWliWW7k,4LxwqKEPKxx4IavfyY2Eqy,1bibYQtOlKNPqk3oA8HGP3,661FdAC9WdjuQxf1sH5PrA,5I2qACyvSTaasSNaWiT90G,2p3RJuPCuNhGNDQEtTPfUQ,1jG1KuT4WsRTaRFGNepUNh
59RNNqeEfkq3X5pfOQxZ3C,4JdxxPk2G1IUAZuiBU0Ky0,77YpxagBmCQV9cVzSi284F,3lhyS1qBiQRFK6H5xeLhgM,0lPCROErEeqqqzwRMMXJHR,7BdyDHXrhVcMUaAtrkorqB,2X3Nb0mIGuM6UT333nSkxJ,3fTET3MXmzX8cu0rp8bp0z,0muopq9ox1UYctxg3lmntI,0kRTxpOdgb1trhSTH56ckF,2RNogAZbrE1oWAw7N6gK2l
...,...,...,...,...,...,...,...,...,...,...
2i9uaNzfUtuApAjEf1omV8,7ry34dFOuoK2c5GoYWWsl3,3fVFXcRvNI7Xm8ZT0uWFb4,0CsLshwxsSHYq3m6gLw3Uo,3JBB92odunFru2gOvI91x9,6YKLlLoEYRujlAH8SgkTge,1BzdE75pSHlsqF2roxxEWk,1T9MOWPyxsDfZEGhTZgnpv,5byJBGDhkVXs81MS3b3Ncq,6a5xI87SEuxoeX6riFYEjM,6dHrWj35HXTGvlCasE5VAA
1svyqzg4SYrPTGqPnpZf9y,2X33IKOkJqn569wiEFJcFF,2QrA62LLiEVHtuuBVkxLe3,4fptYpDyXXNUPtwZ4u8r0R,3HTJiXBvOeRflzJJXXnKnJ,457P4mpRIBEu0qnFOVlS3J,2EGj5Sg6Kqf6oOedLbjzaf,3BYpu3oDuWFXnaMsCH1TBE,5UZBnxh2cz9ByXlpJJTQet,7vFVlFucwHvIEvMZx8Sey2,3v06UNK6kCvjZZhhTaCbQk
4VWnuR2hpj6DZNfcpe4e3M,0DllReoLJDFRClcmIAy5zv,7CmVyZFVIqLpT6LKpa1T7B,1YhcRGVpRW2nywfuWzwBGi,46O8iNdlQHZf7L5JsAKgvi,6xEO56fFPkoAy8WMFxmH2w,5zD3p4ajDSNlAHgzVJ6oWv,2U1DQrNwkkotZxgyW3sZVu,38ujL5gyUYLF0lh4aHXoWh,6fOKU0tquifCX7pLlAClXP,4zJ6t4wLwUhupMSwOv5Cmc
5IUYCeG5ORYA8pNS2Yrqxq,38ujL5gyUYLF0lh4aHXoWh,None,None,None,None,None,None,None,None,None


In [20]:
import json

# Convert the storage variable to JSON format
storage_json = json.dumps(storage)

# Specify the file path where you want to save the JSON file
file_path = 'artists_top_songs.json'

# Write the JSON data to the file
with open(file_path, 'w') as file:
    file.write(storage_json)

print(f"Storage data has been saved as {file_path}")


Storage data has been saved as artists_top_songs.json


In [40]:
id = sp.artist_top_tracks(artist_id='5szilpXHcwOqnyKLqGco5j',
                                    country='US')
for track in id['tracks']:
    print(track['id'])
    print(track['name'])



KeyboardInterrupt: 